In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta

In [2]:
df_pararius = pd.read_csv('app/df_coo_pararius.csv',index_col=[0])
house_temp  = pd.read_csv('data/processed/house_temp.csv',index_col=[0])

In [3]:
temp1 = df_pararius['url'].unique()
temp2 = house_temp['url'].unique()
urls = list(set(temp1) - set(temp2))
print(len(urls))

236


In [4]:
def key_val(text):
    part = html_soup.find("dt",text=text)
    key = (part.text.strip())
    val = (part.findNext("dd").text.strip())
    return key, val

In [5]:
not_available=[]
description_2=[]

for url in urls:
    response = get(url)
    description_1={}
    html_soup = BeautifulSoup(response.text,'html.parser')
    try:
        key1, val1 = key_val("Offered since")
        key2, val2 = key_val("Available")
        description_1.update({
            'url':url,
            key1:val1,
            key2:val2,
        })
        description_2.append(description_1)
    except:
        pass

In [6]:
house_temp_ = pd.DataFrame(description_2)

In [7]:
house_temp = pd.concat([house_temp, house_temp_],0)

<ipython-input-7-9b14ce7ced2d>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  house_temp = pd.concat([house_temp, house_temp_],0)


In [8]:
def fixing_time_delta(time,n):
    fixing_time_delta = house_temp.loc[house_temp['Offered since'].str.contains(time, na=False), 'Offered since']
    fixing_time_delta = fixing_time_delta.str.replace("\D","",regex=True)
    fixing_time_delta = fixing_time_delta.astype(int)
    fixing_time_delta = fixing_time_delta*n
    fixing_time_delta = datetime.today() - fixing_time_delta.map(dt.timedelta)
    house_temp.loc[house_temp['Offered since'].str.contains(time, na=False), 'Offered since'] = fixing_time_delta
    return house_temp

In [9]:
house_temp = fixing_time_delta("week",7)
house_temp = fixing_time_delta("month",30)

In [10]:
house_temp

,url,Offered since,Available
0,https://www.pararius.com/apartment-for-rent/ma...,2021-09-16 09:38:56.811490,Immediately
1,https://www.pararius.com/apartment-for-rent/de...,29-09-2021,From 01-11-2021
2,https://www.pararius.com/apartment-for-rent/ro...,12-10-2021,Immediately
3,https://www.pararius.com/apartment-for-rent/ro...,12-10-2021,From 01-12-2021
4,https://www.pararius.com/house-for-rent/purmer...,12-10-2021,From 01-12-2021
...,...,...,...
231,https://www.pararius.com/apartment-for-rent/am...,13-10-2021,From 02-11-2021
232,https://www.pararius.com/apartment-for-rent/vo...,13-10-2021,In consultation
233,https://www.pararius.com/apartment-for-rent/di...,14-10-2021,In consultation
234,https://www.pararius.com/apartment-for-rent/we...,13-10-2021,From 01-11-2021


In [11]:
house_temp['Offered since'] = pd.to_datetime(house_temp['Offered since']).dt.strftime('%Y-%m-%d') #, format="%Y-%m-%d")

In [12]:
result = pd.merge(df_pararius,house_temp,how='left',on='url')

In [22]:
result2 = result[[
    'deal',
    'garden-surface-area',
    'img',
    'price',
    'link',
    'agency',
    'surface-area',
    'interior',
    'latitude',
    'longitude',
    'number-of-rooms',
    'Plaats',
    'url',
    'date',
    'status',
    'irl',
    'image',
    'address',
    'street',
    'Offered since',
    'Available'
]]


result2.to_csv('app/df_coo_pararius2.csv')
result2

,deal,garden-surface-area,img,price,link,agency,surface-area,interior,latitude,longitude,...,Plaats,url,date,status,irl,image,address,street,Offered since,Available
0,0.037093,0.0,<a href=https://www.pararius.com/apartment-for...,1995.0,<a target='_blank' href=https://www.pararius.c...,Fransen Vastgoed,72,Furnished,51.916296,4.489444,...,Rotterdam,https://www.pararius.com/apartment-for-rent/ro...,2021-10-14,Free,/apartment-for-rent/rotterdam/b2788bae/boompjes,https://media.pararius.nl/image/PR0001658000/P...,3011 XB Rotterdam (Stadsdriehoek),Apartment Boompjes,2021-09-09,Immediately
1,0.061599,0.0,<a href=https://www.pararius.com/apartment-for...,2500.0,<a target='_blank' href=https://www.pararius.c...,Sonar appartementen,150,Upholstered,52.112566,4.282071,...,Den Haag,https://www.pararius.com/apartment-for-rent/de...,2021-10-14,Free,/apartment-for-rent/den-haag/7f14327f/gevers-d...,https://casco-media-prod.global.ssl.fastly.net...,2586 CR Den Haag (Scheveningen Badplaats),Apartment Gevers Deynootplein,2021-09-29,From 01-11-2021
2,0.053331,0.0,<a href=https://www.pararius.com/apartment-for...,600.0,<a target='_blank' href=https://www.pararius.c...,Divorce Housing,30,Upholstered,52.177358,4.461527,...,Oegstgeest,https://www.pararius.com/apartment-for-rent/oe...,2021-10-14,Free,/apartment-for-rent/oegstgeest/e35c50e6/rhijng...,https://media.pararius.nl/image/PR0001659000/P...,2342 AN Oegstgeest (Rhijngeest),Apartment Rhijngeesterstraatweg,2021-10-14,Immediately
3,0.079047,0.0,<a href=https://www.pararius.com/apartment-for...,1050.0,<a target='_blank' href=https://www.pararius.c...,P&D Housing,80,Upholstered,51.558725,5.072332,...,Tilburg,https://www.pararius.com/apartment-for-rent/ti...,2021-10-14,Free,/apartment-for-rent/tilburg/6b8bc5f4/boomstraat,https://casco-media-prod.global.ssl.fastly.net...,5038 GV Tilburg (Bomenbuurt Midden),Apartment Boomstraat,2021-10-14,From 01-11-2021
4,0.042856,0.0,<a href=https://www.pararius.com/apartment-for...,1050.0,<a target='_blank' href=https://www.pararius.c...,Kimmel & Co. Makelaarskantoor o.g. b.v.,43,Upholstered,52.080994,4.290151,...,Den Haag,https://www.pararius.com/apartment-for-rent/de...,2021-10-14,Free,/apartment-for-rent/den-haag/b2db1de5/roggevee...,https://casco-media-prod.global.ssl.fastly.net...,2518 TT Den Haag (Zeeheldenkwartier),Apartment Roggeveenstraat 140,2021-10-14,Immediately
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5523,0.059999,0.0,<a href=https://www.pararius.com/house-for-ren...,1500.0,<a target='_blank' href=https://www.pararius.c...,Uw Partner in Onroerende Zaken,86,Upholstered,52.432377,4.838417,...,Zaandam,https://www.pararius.com/house-for-rent/zaanda...,2021-10-14,Under option,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,2021-07-16,Immediately
5524,0.086956,0.0,<a href=https://www.pararius.com/house-for-ren...,3450.0,<a target='_blank' href=https://www.pararius.c...,BenHousing,294,Upholstered or furnished,51.940053,4.455394,...,Rotterdam,https://www.pararius.com/house-for-rent/rotter...,2021-10-14,Under option,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,2021-07-16,Immediately
5525,0.097915,0.0,<a href=https://www.pararius.com/apartment-for...,960.0,<a target='_blank' href=https://www.pararius.c...,Regio Vastgoedmakelaars,90,Shell,52.175591,5.283004,...,Soest,https://www.pararius.com/apartment-for-rent/so...,2021-10-14,Under option,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,2021-07-16,Immediately
5526,0.058518,0.0,<a href=https://www.pararius.com/apartment-for...,1350.0,<a target='_blank' href=https://www.pararius.c...,Uw Partner in Onroerende Zaken,76,Upholstered,52.494378,4.756011